In [1]:
import numpy as np
import libpysal
import spreg
import geopandas as gpd
import polars as pl
from local_functions import *

In [2]:
territorial_information = gpd.read_file("data/transformed/territorial_information_summary.geojson")

In [3]:
for c in territorial_information.columns:
    print(c)

Teritoriālā vienība
Platība (km^2)
Gads
Iedzīvotāju skaits gada sākumā
Gada vidējais iedzīvotāju skaits
Iedzīvotāju dabiskais pieaugums
Migrācijas saldo
Iedzīvotāju skaita izmaiņas - pavisam
Mēneša darba samaksas mediāna
Iedzīvotāji darbspējas vecumā
Pašvaldības izdevumi: Vispārējie valdības dienesti (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Vispārējie valdības dienesti (Kapitālie izdevumi)
Pašvaldības izdevumi: Sabiedriskā kārtība un drošība (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Sabiedriskā kārtība un drošība (Kapitālie izdevumi)
Pašvaldības izdevumi: Ekonomiskā darbība (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Teritoriju un mājokļu apsaimniekošana (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Teritoriju un mājokļu apsaimniekošana (Kapitālie izdevumi)
Pašvaldības izdevumi: Atpūta, kultūra un reliģija (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Atpūta, kultūra un reliģija (Kapitālie izdevumi)
Pašvaldības izdevumi: Izglītība (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Izgl

In [17]:
y_col = ['Pašvaldības ieņēmumi: Ieņēmumi no iedzīvotāju ienākuma nodokļa (uz 1 iedz.)']
x_col = [
    'Mēneša darba samaksas mediāna',
    'Nodarbinātības līmenis',
    'Iedzīvotāji darbspējas vecumā (%)',
    'Ekonomiskās aktivitātes līmenis',
    'Pašvaldības izdevumi: Ekonomiskā darbība (Kapitālie izdevumi) (uz 1 iedz.)',
    'Ekonomiski aktīvi uzņēmumi (uz 1000 iedz.)',
    'Pašvaldības izdevumi: Izglītība (Uzturēšanas izdevumi) (uz 1 iedz.)',
    'Ceļu un ielu izdevumi (investīcijas) (uz 1 iedz.)',
    'Migrācijas saldo (%)',
    'Iedzīvotāju īpatsvars ar augstāko izglītību'
]

columns_to_use = [
    'Teritoriālā vienība',
    'Gads',
    'geometry'
] + x_col + y_col

In [18]:
(
    pl.from_dataframe(territorial_information.drop("geometry", axis=1))
    .unpivot(
        on=y_col + x_col,
        index=['Teritoriālā vienība', 'Gads']
    )
    .filter(~pl.col("value").is_null())
    .group_by("variable")
    .agg(
        pl.col("Gads").min().alias("Min year"),
        pl.col("Gads").max().alias("Max year")
    )
    .group_by(1)
    .agg(
        pl.col("Min year").max(),
        pl.col("Max year").min()
    )
)

literal,Min year,Max year
i32,i32,i32
1,2021,2023


In [19]:
df = territorial_information[columns_to_use]
df = df[df.Gads.isin([2021, 2022, 2023])]
df = df.fillna(0)
df = df.sort_values(by=['Teritoriālā vienība', 'Gads'])

In [20]:
t = df[df.Gads==2023]
w = libpysal.weights.Queen.from_dataframe(t, ids="Teritoriālā vienība")
w.transform = 'r'

In [21]:
df[x_col]

,Mēneša darba samaksas mediāna,Nodarbinātības līmenis,Iedzīvotāji darbspējas vecumā (%),Ekonomiskās aktivitātes līmenis,Pašvaldības izdevumi: Ekonomiskā darbība (Kapitālie izdevumi) (uz 1 iedz.),Ekonomiski aktīvi uzņēmumi (uz 1000 iedz.),Pašvaldības izdevumi: Izglītība (Uzturēšanas izdevumi) (uz 1 iedz.),Ceļu un ielu izdevumi (investīcijas) (uz 1 iedz.),Migrācijas saldo (%),Iedzīvotāju īpatsvars ar augstāko izglītību
323,838.0,51.7,0.621378,56.6,185.103347,67.0,650.161474,5.205673,0.000477,21.3
366,944.0,51.0,0.616452,55.5,80.784374,69.0,628.043332,76.220306,0.005989,21.9
409,1077.0,50.7,0.612850,55.1,95.787599,68.0,786.987807,69.284318,-0.002176,22.1
301,716.0,52.6,0.616478,58.8,49.767838,77.0,553.604574,25.794748,-0.006926,21.5
344,817.0,52.2,0.615617,58.2,236.518729,82.0,619.892862,184.962395,-0.008553,22.2
...,...,...,...,...,...,...,...,...,...,...
375,1273.0,63.4,0.621764,66.1,0.465230,88.0,734.185335,79.774152,0.037049,46.2
418,1440.0,63.2,0.621029,65.8,2.347291,85.0,827.251988,27.032163,0.022453,46.8
338,1099.0,64.1,0.619044,67.8,110.941450,81.0,578.944077,80.876384,0.013831,39.8
381,1228.0,63.2,0.618141,66.3,93.966395,86.0,655.583391,41.494519,0.018389,40.8


In [22]:
Y_df = df[y_col+["Teritoriālā vienība"]].groupby("Teritoriālā vienība")[y_col].apply(lambda df: df.reset_index(drop=True)).unstack()

In [23]:
X_df = df[x_col+["Teritoriālā vienība"]].groupby("Teritoriālā vienība")[x_col].apply(lambda df: df.reset_index(drop=True)).unstack()

In [24]:
Y = Y_df.to_numpy()
X = X_df.to_numpy()

fe_lag = spreg.Panel_FE_Lag(Y, X, w, name_y = y_col, name_x=x_col, name_ds="df")

In [25]:
print(fe_lag.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG PANEL - FIXED EFFECTS
-----------------------------------------------------------------------
Data set            :          df
Weights matrix      :     unknown
Dependent Variable  :Pašvaldības ieņēmumi: Ieņēmumi no iedzīvotāju ienākuma nodokļa (uz 1 iedz.)                Number of Observations:         129
Mean dependent var  :      0.0000                Number of Variables   :          11
S.D. dependent var  :     84.3424                Degrees of Freedom    :         118
Pseudo R-squared    :      0.8709
Spatial Pseudo R-squared:  0.8723
Sigma-square ML     :     911.431                Log likelihood        :    -936.159
S.E of regression   :      30.190                Akaike info criterion :    1894.317
                                                 Schwarz criterion     :    1925.775

------------------------------------------------------------------------------------
            Variable     Coefficient   